In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix,new_state

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0.4],dtype = np.complex128)
zeta = np.array([0.2,0.3],dtype = np.complex128)
theta = 0.2
psi = 0
theta1 = 0.3
psi1 = 0.5
cutoff_dim = 4
cutoff= 4
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [4]:
old_state

array([[0.25646067+0.18696159j, 0.10957359+0.14690812j,
        0.0059518 +0.14287565j, 0.14381896+0.0526344j ],
       [0.14010729+0.25615837j, 0.16729974+0.29805162j,
        0.04843372+0.01635974j, 0.04860981+0.13526358j],
       [0.07282036+0.08211066j, 0.19459576+0.02540385j,
        0.20628883+0.30258065j, 0.30492402+0.00277154j],
       [0.23485284+0.25819264j, 0.02506248+0.28978616j,
        0.19726235+0.10217667j, 0.16678155+0.18759449j]])

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [6]:
mu

array([ 0.+0.j,  0.+0.j, -0.-0.j, -0.-0.j])

In [7]:
Sigma

array([[ 0.19388392-0.02140784j, -0.04719474+0.05186019j,
        -0.86858571-0.02692664j,  0.42921212-0.1328333j ],
       [-0.04719474+0.05186019j,  0.18131823+0.20751107j,
        -0.3437266 -0.29311592j, -0.78920265-0.30371088j],
       [-0.86858571-0.02692664j, -0.3437266 -0.29311592j,
        -0.20108298+0.j        , -0.01829045+0.j        ],
       [ 0.42921212-0.1328333j , -0.78920265-0.30371088j,
        -0.01829045+0.j        , -0.28760495+0.j        ]])

In [8]:
G_00pq

array([[0.96840546+0.j, 0.        -0.j, 0.19694211+0.j, 0.        -0.j],
       [0.        -0.j, 0.01771257+0.j, 0.        -0.j, 0.00623912+0.j],
       [0.1376948 +0.j, 0.        -0.j, 0.02832661+0.j, 0.        -0.j],
       [0.        -0.j, 0.00436217+0.j, 0.        -0.j, 0.00154247+0.j]])

In [10]:
# R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)
import math
sqrtfac = np.zeros(5)
for i in range(5):
    sqrtfac[i] = np.sqrt(math.factorial(i+1))

sqrtsfac = np.zeros((cutoff, cutoff), dtype=dtype)
for m in range(cutoff):
    for n in range(cutoff):
        sqrtsfac[m,n]= np.sqrt(math.factorial(m+1))*np.sqrt(math.factorial(n+1))
        
for j in range(cutoff):
    for k in range(cutoff):
        if j==0 and k ==0:
            R[0,0,j,k] = np.sum(G_00pq*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G_00pq[:cutoff-j,:] * sqrtfac[:cutoff-j,None]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[:,k:] = G_00pq[:,:cutoff-k] * sqrtfac[:cutoff-k]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,k:] = G_00pq[:cutoff-j,:cutoff-k] * sqrtsfac[:cutoff-j,:cutoff-k]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

#R00jk is correct
    
#R_0n^jk
for n in range(1,cutoff):
    for k in range(0,cutoff): 
        for j in range(0,cutoff):
         
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]
#R0100 correct but others are not correct
print(R[0,1,:cutoff,:cutoff])

for m in range(1,cutoff):
    for n in range(0,cutoff):
        for j in range(0,cutoff):
            for k in range(0,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


[[ 0.06920211+0.36673012j -0.00891898+0.44245657j  0.06711884+0.30496115j
  -0.08122957+0.11129308j]
 [ 0.14232026+0.49394494j  0.1002415 +0.31457425j  0.00113089+0.34821867j
   0.06230258+0.08468119j]
 [ 0.24303033+0.36640476j  0.24246959+0.29250186j  0.23255155+0.17855262j
  -0.02632254+0.07103304j]
 [ 0.18212006+0.25475456j  0.16554461+0.07623236j -0.00228985+0.14178773j
   0.        +0.j        ]]


In [11]:
#example for R_01^01
-Sigma[1,2]*R[0,0,1,1]-Sigma[1,3]*R[0,0,0,2]

(-0.00891898143739571+0.4424565743457287j)

In [12]:
#example for R_01^01
j=0
k=1
G22 = G[0,1,:,:]
a = np.zeros((cutoff,cutoff),dtype=dtype)
if j==0 and k ==0:
    a[j,k] = np.sum(G22*old_state)

elif j>0 and k==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,:] = G22[:cutoff-j,:] * sqrtfac[:cutoff-j,None]
    a[j,k]  = np.sum(G_00pq2*old_state)

elif k>0 and j==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[:,k:] = G22[:,:cutoff-k] * sqrtfac[:cutoff-k]
    a[j,k]  = np.sum(G_00pq2*old_state)
else:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,k:] = G22[:cutoff-j,:cutoff-k]  * sqrtsfac[:cutoff-j,:cutoff-k]
    a[j,k]  = np.sum(G_00pq2*old_state)
print(a[j,k])


(-0.021782361058736835+0.5214364640411997j)


In [13]:
#Use g_matrix to get the R matrix
m=0
n=1
G22 = G[m,n,:,:]
a = np.zeros((cutoff,cutoff),dtype=dtype)
for j in range(cutoff):
    for k in range(cutoff):
        if j==0 and k ==0:
            a[j,k] = np.sum(G22*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G22[:cutoff-j,:] * sqrtfac[:cutoff-j,None]
            a[j,k]  = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[:,k:] = G22[:,:cutoff-k] * sqrtfac[:cutoff-k]
            a[j,k]  = np.sum(G_00pq2*old_state)
        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,k:] = G22[:cutoff-j,:cutoff-k]  * sqrtsfac[:cutoff-j,:cutoff-k]
            a[j,k]  = np.sum(G_00pq2*old_state)
print("R_01",a)# a = R[m,n,:,:]

R_01 [[ 0.05378589+0.31831339j -0.02178236+0.52143646j  0.09556302+0.38502722j
  -0.07854439+0.10404696j]
 [ 0.15058486+0.50185608j  0.15056992+0.4516372j   0.00060246+0.49514568j
   0.08810915+0.11975728j]
 [ 0.23078792+0.42225831j  0.34281706+0.41902979j  0.32887755+0.25251154j
  -0.03722569+0.10045589j]
 [ 0.18245416+0.2340672j   0.23411544+0.10780884j -0.00323834+0.20051814j
   0.        +0.j        ]]


In [14]:
R[0,1,:cutoff,:cutoff]

array([[ 0.06920211+0.36673012j, -0.00891898+0.44245657j,
         0.06711884+0.30496115j, -0.08122957+0.11129308j],
       [ 0.14232026+0.49394494j,  0.1002415 +0.31457425j,
         0.00113089+0.34821867j,  0.06230258+0.08468119j],
       [ 0.24303033+0.36640476j,  0.24246959+0.29250186j,
         0.23255155+0.17855262j, -0.02632254+0.07103304j],
       [ 0.18212006+0.25475456j,  0.16554461+0.07623236j,
        -0.00228985+0.14178773j,  0.        +0.j        ]])

In [15]:
#Result from recursive process
R[:,:,0,0]

array([[ 0.07015205+0.23347341j,  0.06920211+0.36673012j,
        -0.1425196 +0.35432212j, -0.0655645 +0.07926995j],
       [ 0.22631156+0.31101653j,  0.07079101+0.24915387j,
         0.00645266+0.1077181j ,  0.00600983+0.0908864j ],
       [-0.15890934-0.06001001j,  0.07323268+0.03603007j,
         0.07148838+0.00832972j,  0.02681953-0.02160199j],
       [-0.09923173+0.01188942j, -0.08304045-0.01048561j,
         0.00566737-0.054486j  ,  0.03603313-0.02318173j]])

In [5]:
new_state(gamma, phi, theta1, psi1, zeta, theta, psi, old_state)

array([[ 0.26903369+0.2367468j ,  0.08897821+0.36421038j,
        -0.24700811+0.28289314j,  0.01855737+0.16486207j],
       [ 0.07371647+0.23826688j,  0.01556685+0.25404751j,
        -0.0168841 +0.18982934j, -0.22199595+0.12239678j],
       [-0.10735573-0.08220541j,  0.07918438+0.20259116j,
        -0.08208314+0.00806294j,  0.00836761-0.0547479j ],
       [-0.02211391+0.16546846j, -0.06430235-0.1290079j ,
        -0.0275257 -0.14694086j,  0.06604169-0.05331375j]])

In [6]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [7]:
#Result from G matrix
np.einsum("cd,abcd->ab",old_state,G)

array([[ 0.26903369+0.2367468j ,  0.08897821+0.36421038j,
        -0.24700811+0.28289314j,  0.01855737+0.16486207j],
       [ 0.07371647+0.23826688j,  0.01556685+0.25404751j,
        -0.0168841 +0.18982934j, -0.22199595+0.12239678j],
       [-0.10735573-0.08220541j,  0.07918438+0.20259116j,
        -0.08208314+0.00806294j,  0.00836761-0.0547479j ],
       [-0.02211391+0.16546846j, -0.06430235-0.1290079j ,
        -0.0275257 -0.14694086j,  0.06604169-0.05331375j]])

In [8]:
np.sum(G[0,1,:,:]*old_state)

(0.08897820680882346+0.3642103812866662j)